In [1]:
pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 129.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 33.6 

In [2]:
!pip install flask torch torchaudio TTS pyngrok flask-cors


In [7]:
# import os
# import torch
# import torchaudio
# from flask import Flask, request, send_file, jsonify
# from flask_cors import CORS
# from TTS.api import TTS
# from pyngrok import ngrok


# UPLOAD_FOLDER = "./uploads"
# OUTPUT_FOLDER = "./outputs"
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# app = Flask(__name__)
# CORS(app)  # Enable CORS to allow frontend requests from different origins

# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
# app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

# # Set your ngrok auth token here
# ngrok.set_auth_token("2t5jG4Kb85xObYFT6mzyG088vu1_61sQEm5o3GesZbmChuQHR")
# public_url = ngrok.connect(5000)
# print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# # Load TTS model and move to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2").to(device)
# print("TTS model loaded on", device)

# @app.route('/')
# def home():
#     return jsonify({"message": "Backend is running."})

# @app.route('/process', methods=['POST'])
# def process():

#     if 'voice_sample' not in request.files:
#         return jsonify({"error": "No voice sample provided."}), 400
#     file = request.files['voice_sample']
#     if file.filename == '':
#         return jsonify({"error": "No file selected."}), 400

#     # Validate text input
#     text = request.form.get('text')
#     if not text:
#         return jsonify({"error": "Text input is required."}), 400


#     input_filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
#     file.save(input_filepath)

#     # Load the audio file and resample if needed
#     try:
#         waveform, sample_rate = torchaudio.load(input_filepath)
#     except Exception as e:
#         return jsonify({"error": f"Error loading audio file: {str(e)}"}), 400

#     if sample_rate != 22050:
#         try:
#             resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=22050)
#             waveform = resampler(waveform)
#         except Exception as e:
#             return jsonify({"error": f"Error during resampling: {str(e)}"}), 400

#     # Save the cleaned voice file for TTS processing
#     cleaned_filename = "cleaned_" + file.filename
#     cleaned_filepath = os.path.join(app.config['UPLOAD_FOLDER'], cleaned_filename)
#     torchaudio.save(cleaned_filepath, waveform, 22050)

#     # Define the output file path
#     output_filepath = os.path.join(app.config['OUTPUT_FOLDER'], "output_speech.wav")

#     try:

#         tts.tts_to_file(text=text, speaker_wav=cleaned_filepath, language="en", file_path=output_filepath)
#     except Exception as e:
#         return jsonify({"error": f"Error generating speech: {str(e)}"}), 500

#     # Return the generated file as an attachment
#     return send_file(output_filepath, as_attachment=True)

# if __name__ == '__main__':
#     app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://daaf-34-125-100-225.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
TTS model loaded on cuda
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 > Text splitted to sentences.
['Amid an ongoing language row in the country, Maharashtra Chief Minister Devendra Fadnavis firmly asserted in the legislative assembly that Marathi is the official language of the state and that everyone residing in the state should learn it.', "Emphasising the language's significance, he stated that Marathi is an integral part of the state's culture and identity, making it the duty of every citizen to learn and respect it."]


INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 07:08:23] "POST /process HTTP/1.1" 200 -


 > Processing time: 13.252866268157959
 > Real-time factor: 0.42385951092608926


In [8]:
import os
import torch
import torchaudio
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from TTS.api import TTS
from pyngrok import ngrok

# Set up folders
UPLOAD_FOLDER = "./uploads"
OUTPUT_FOLDER = "./outputs"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for frontend requests

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

# Ngrok setup (use your own auth token)
ngrok.set_auth_token("2t5jG4Kb85xObYFT6mzyG088vu1_61sQEm5o3GesZbmChuQHR")
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Load TTS model (supports multilingual and emotions)
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2").to(device)
print("TTS model loaded on", device)

# Define emotion mapping (can be expanded)
EMOTION_MAP = {
    "neutral": "neutral",
    "happy": "happy",
    "sad": "sad",
    "angry": "angry",
    "fearful": "fearful",
    "excited": "excited",
    "disgusted": "disgusted",
    "surprised": "surprised"
}

@app.route('/')
def home():
    return jsonify({"message": "Backend is running with emotional intelligence."})

@app.route('/process', methods=['POST'])
def process():
    if 'voice_sample' not in request.files:
        return jsonify({"error": "No voice sample provided."}), 400
    file = request.files['voice_sample']
    if file.filename == '':
        return jsonify({"error": "No file selected."}), 400

    text = request.form.get('text')
    if not text:
        return jsonify({"error": "Text input is required."}), 400

    emotion = request.form.get('emotion', 'neutral').lower()
    if emotion not in EMOTION_MAP:
        return jsonify({"error": "Invalid emotion selected."}), 400

    # Save uploaded file
    input_filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(input_filepath)

    # Load and resample audio if needed
    try:
        waveform, sample_rate = torchaudio.load(input_filepath)
    except Exception as e:
        return jsonify({"error": f"Error loading audio file: {str(e)}"}), 400

    if sample_rate != 22050:
        try:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=22050)
            waveform = resampler(waveform)
        except Exception as e:
            return jsonify({"error": f"Error during resampling: {str(e)}"}), 400

    # Save cleaned voice sample
    cleaned_filename = "cleaned_" + file.filename
    cleaned_filepath = os.path.join(app.config['UPLOAD_FOLDER'], cleaned_filename)
    torchaudio.save(cleaned_filepath, waveform, 22050)

    # Define output path
    output_filepath = os.path.join(app.config['OUTPUT_FOLDER'], "output_speech.wav")

    try:
        # Generate speech with emotion
        tts.tts_to_file(
            text=text,
            speaker_wav=cleaned_filepath,
            language="en",
            file_path=output_filepath,
            emotion=EMOTION_MAP[emotion]
        )
    except Exception as e:
        return jsonify({"error": f"Error generating speech: {str(e)}"}), 500

    return send_file(output_filepath, as_attachment=True)

if __name__ == '__main__':
    app.run(port=5000)



 * ngrok tunnel "NgrokTunnel: "https://a9a2-34-125-100-225.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
TTS model loaded on cuda
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 > Text splitted to sentences.
['Amid an ongoing language row in the country, Maharashtra Chief Minister Devendra Fadnavis firmly asserted in the legislative assembly that Marathi is the official language of the state and that everyone residing in the state should learn it.', "Emphasising the language's significance, he stated that Marathi is an integral part of the state's culture and identity, making it the duty of every citizen to learn and respect it."]


INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 07:12:08] "POST /process HTTP/1.1" 200 -


 > Processing time: 14.31372618675232
 > Real-time factor: 0.42599455580525286
